# STACL机器同传
机器同传，即在句子完成之前进行翻译，同声传译的目标是实现同声传译的自动化，它可以与源语言同时翻译，延迟时间只有几秒钟。

同传翻译的难点在于源语言和目标语言之间词序的差异带来的翻译延迟。 例如，考虑将SOV（主宾谓）语言（如日语或德语）翻译为SVO（主谓宾）语言（如英语或汉语），必须等到源语言动词出现才可以准确翻译。因此，翻译系统必须求助于传统的全句翻译，因此造成至少一句话的延迟。

本项目是基于机器翻译领域主流模型 Transformer网络结构的同传模型STACL的PaddlePaddle 实现，包含模型训练，预测以及使用自定义数据等内容。用户可以基于发布的内容搭建自己的同传翻译模型。

STACL 是论文 [STACL: Simultaneous Translation with Implicit Anticipation and Controllable Latency using Prefix-to-Prefix Framework](https://www.aclweb.org/anthology/P19-1289/) 中针对同传提出的适用于所有同传场景的翻译架构，该架构基于Transformer实现，可参考PaddleNLP的[Transformer](https://github.com/PaddlePaddle/PaddleNLP/tree/develop/examples/machine_translation/transformer)。

STACL 主要具有以下优势：

- **Implicit Anticipation**: Prefix-to-Prefix架构拥有预测能力，即在未看到源词的情况下仍然可以翻译出对应的目标词，克服了SOV→SVO等词序差异；

- **Controllable Latency**: Wait-k策略可以不需要全句的源句，直接预测目标句，可以实现任意的字级延迟，同时保持较高的翻译质量。

### Implicit Anticipation（隐式的预测能力）
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/9243f9077b16417c9148333b731ac5d2275cf79ef5df4579af5ff644af5b9d47" width="1000" height="500" ></center>

<br><center>图1：Implicit Anticipation</center></br>
源端只输入“他 还 说 现在 正在 为 这 一 会议”，目标端前两个策略就已经翻译出“making preparations”和“making”，充分体现了STACL的预测能力。

### Controllable Latency（可控的延迟）
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/8c919fe235d6415b97ffe91c54bc0ab991d3cfac5b714b6f90a32d40cf34983f" width="1000" height="500" ></center>
<br><center>图2： Controllable Latency (Wait-k)</center></br>
Wait-k策略首先等待源端读入k个词后开始进行翻译。上图2中，k=1，第一个目标词在读入第一个1个源词后翻译，第二个目标词在读入前2个源词后翻译，以此类推，所以当源端读入“他 还 说”3个词后，目前端就已经翻译出“he also said”。当k=3，第一个目标词在读入前3个源词后翻译，所以当源端读入“他 还 说”后，目标端翻译出第一个词”he“。  


# Demo动画展示
如果有ASR(Automatic Speech Recognition)的api接入，即刻实现语音同传


该Demo的具体实现：[STACL Demo](https://github.com/PaddlePaddle/PaddleNLP/tree/develop/examples/simultaneous_translation/stacl/demo)   
<center><img src="https://gongel.cn/wp-content/uploads/2021/05/textdemo_show_20210609.gif" width="1000" height="500" ></center>
<br><center>图3.1：文本同传Demo </center></br>

<center><img src="https://gongel.cn/wp-content/uploads/2021/05/audiodemo_show_20210609.gif" width="1000" height="500" ></center>
<br><center>图3.2：语音同传Demo </center></br>



# 环境介绍

- PaddlePaddle框架，AI Studio平台已经默认安装最新版2.1。

- PaddleNLP，深度兼容框架2.1，是飞桨框架2.1在NLP领域的最佳实践。

**记得给[PaddleNLP](https://github.com/PaddlePaddle/PaddleNLP)点个小小的Star⭐**

开源不易，希望大家多多支持~ 

GitHub地址：[https://github.com/PaddlePaddle/PaddleNLP](https://github.com/PaddlePaddle/PaddleNLP)  
PaddleNLP文档：[https://paddlenlp.readthedocs.io](https://paddlenlp.readthedocs.io)  
本项目完整版：[https://github.com/PaddlePaddle/PaddleNLP/tree/develop/examples/simultaneous_translation/stacl](https://github.com/PaddlePaddle/PaddleNLP/tree/develop/examples/simultaneous_translation/stacl)

![](https://ai-studio-static-online.cdn.bcebos.com/4afc34d18d6940af8ff6139fb0e6aa4ae46fcfd0d8cf49a18b491a91a22f959e)

In [1]:
%cd stacl/

/home/aistudio/stacl


In [2]:
# 安装依赖
!pip install --upgrade paddlenlp -i https://pypi.org/simple
!pip install -r requirements.txt

     |████████████████████████████████| 460kB 25kB/s eta 0:00:013
  Found existing installation: paddlenlp 2.0.1
    Uninstalling paddlenlp-2.0.1:
      Successfully uninstalled paddlenlp-2.0.1
Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 645kB 11.2MB/s eta 0:00:01
  Found existing installation: PyYAML 5.1.2
    Uninstalling PyYAML-5.1.2:
      Successfully uninstalled PyYAML-5.1.2


# Pipeline
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/b7aafbcc8e2f4bfc9b864d1a1bc0af749260ef9b690c458399f2ba9c66c1ab80" width="1200" height="600" ></center>
<br><center>图4：Pipeline </center></br>

In [3]:
import os
import time
import yaml
import logging
import argparse
import numpy as np
from pprint import pprint
from attrdict import AttrDict

import numpy as np
from functools import partial
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
import paddle.distributed as dist
from paddle.io import DataLoader, BatchSampler
from paddlenlp.data import Vocab, Pad
from paddlenlp.datasets import load_dataset
from paddlenlp.transformers import WordEmbedding, PositionalEmbedding, position_encoding_init
from paddlenlp.utils.log import logger

from utils import CrossEntropyCriterion, post_process_seq, Decoder 

## 1. 数据预处理
本项目展示的训练数据为NIST的中英demo数据(1000条中英文本对)，同时提供基于全量NIST中英数据训练的预训练模型下载。  
中文需要Jieba+BPE，英文需要BPE  


### BPE(Byte Pair Encoding)
BPE优势：
- 压缩词表；
- 一定程度上缓解OOV(out of vocabulary)问题
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/e7a59d7bab514a6fa17d24a116af0b680fbd664439c948799c0a0541dffd35a2" width="1000" height="500" ></center>
<br><center>图5：learn BPE </center></br>

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/d4b9c48ba7274395af3fbb267c1f9adcba50dd4b147d4258be58999b3b5a198c" width="1000" height="500" ></center>
<br><center>图6：Apply BPE </center></br>

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/968fefdeff604651b0f779191892106a15b27378b4ab48df85dacc2eeef76b59" width="1000" height="500" ></center>
<br><center>图7：Jieba+BPE </center></br>

### 数据格式:

`兵营 是 双@@ 枪 老@@ 大@@ 爷 的 前提 建筑 之一 。	it serves as a prerequisite for Re@@ apers to be built at the Bar@@ rac@@ ks .`  

与文本翻译的数据预处理几乎一致，可以参考[Transformer机器翻译](https://aistudio.baidu.com/aistudio/projectdetail/1918692)

In [4]:
!bash get_data_and_model.sh

Decompress train_dev_test data.
Download model.
--2021-06-21 21:51:54--  https://paddlenlp.bj.bcebos.com/models/stacl/nist_zhen_full_w5.tar.gz
Resolving paddlenlp.bj.bcebos.com (paddlenlp.bj.bcebos.com)... 182.61.200.229, 182.61.200.195, 2409:8c00:6c21:10ad:0:ff:b00e:67d
Connecting to paddlenlp.bj.bcebos.com (paddlenlp.bj.bcebos.com)|182.61.200.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 849689440 (810M) [application/x-gzip]
Saving to: ‘trained_models/nist_zhen_full_w5.tar.gz’

nist_zhen_full_w5.t 100%[===================>] 810.33M  26.7MB/s    in 33s     

2021-06-21 21:52:27 (24.2 MB/s) - ‘trained_models/nist_zhen_full_w5.tar.gz’ saved [849689440/849689440]

Decompress model.
Over.


## 2. 构造Dataloader

下面的`create_data_loader`函数用于创建训练集、验证集所需要的`DataLoader`对象,  
`create_infer_loader`函数用于创建预测集所需要的`DataLoader`对象，   
`DataLoader`对象用于产生一个个batch的数据。下面对函数中调用的`paddlenlp`内置函数作简单说明：
* `paddlenlp.data.Vocab.load_vocabulary`：Vocab词表类，集合了一系列文本token与ids之间映射的一系列方法，支持从文件、字典、json等一系方式构建词表
* `paddlenlp.datasets.load_dataset`：从本地文件创建数据集时，推荐根据本地数据集的格式给出读取function并传入 load_dataset() 中创建数据集
* `paddlenlp.data.Pad`：padding 操作  

具体可参考[PaddleNLP的文档](https://paddlenlp.readthedocs.io/zh/latest/data_prepare/dataset_self_defined.html)

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/0085d53068134546bcb914347774430c3a2c94cd77934c2e90420ac740d16fc7" width="700" height="350" ></center>
<br><center>图8：构造Dataloader的流程 </center></br>


<center><img src="https://ai-studio-static-online.cdn.bcebos.com/3bf38365718346a19c25729bb67e2e6afe8f0bafc61348018d2b9dd60dc9a8bf" width="1000" height="500" ></center>
<br><center>图9：Dataloader细节 </center></br>

In [5]:
# 自定义读取本地数据的方法
def read(src_tgt_file, only_src=False):
    with open(src_tgt_file, 'r', encoding='utf8') as src_tgt_f:
        for line in src_tgt_f:
            line = line.strip('\n')
            if not line:
                continue
            line_split = line.split('\t')
            # 判断是否是测试集，测试集不需要target
            if only_src:
                yield {"src": line_split[0]}
            else:
                if len(line_split) != 2:
                    continue
                yield {"src": line_split[0], "trg": line_split[1]}

 # 过滤掉长度 ≤min_len或者≥max_len 的数据  
def min_max_filer(data, max_len, min_len=0):
    # 获取每条src和tgt的最小长度和最大长度（+1是为了<s>或者<e>），过滤掉不满足长度范围的样本
    data_min_len = min(len(data[0]), len(data[1]))
    data_max_len = max(len(data[0]), len(data[1]))
    return (data_min_len >= min_len) and (data_max_len <= max_len)

In [6]:
# 为训练集和验证集创建dataloader
def create_data_loader(args, places=None):
    data_files = {'train': args.training_file, 'dev': args.validation_file}

    # 通过paddlenlp.datasets.load_dataset从本地文件创建数据集：根据本地数据集的格式给出读取function并传入load_dataset()中创建数据集
    datasets = [
        load_dataset(
            read, src_tgt_file=filename, lazy=False)
        for split, filename in data_files.items()
    ]

    # 通过Paddlenlp.data.Vocab.load_vocabulary从本地创建词表
    src_vocab = Vocab.load_vocabulary(
        args.src_vocab_fpath,
        bos_token=args.special_token[0],
        eos_token=args.special_token[1],
        unk_token=args.special_token[2])
    trg_vocab = Vocab.load_vocabulary(
        args.trg_vocab_fpath,
        bos_token=args.special_token[0],
        eos_token=args.special_token[1],
        unk_token=args.special_token[2])

    args.src_vocab_size = len(src_vocab)
    args.trg_vocab_size = len(trg_vocab)

    def convert_samples(sample):
        source = sample['src'].split()
        target = sample['trg'].split()

        # 将tokens转化为词表对应的ids
        source = src_vocab.to_indices(source) + [args.eos_idx]
        target = [args.bos_idx] + \
                 trg_vocab.to_indices(target) + [args.eos_idx]

        return source, target

    # 训练集dataloader和验证集dataloader  
    data_loaders = []
    for i, dataset in enumerate(datasets):
        # 通过Dataset的map方法将样本token转换为id；通过Dataset的filter方法过滤掉不符合条件的样本
        dataset = dataset.map(convert_samples, lazy=False).filter(
            partial(min_max_filer, max_len=args.max_length))

        # BatchSampler，批采样器BatchSampler组batch: https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/io/BatchSampler_cn.html
        batch_sampler = BatchSampler(dataset,batch_size=args.batch_size, shuffle=True,drop_last=False)

        # DataLoader，构造Dataloader用于后续迭代取数据进行训练/验证/测试: https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/io/DataLoader_cn.html
        data_loader = DataLoader(
            dataset=dataset,
            places=places,
            batch_sampler=batch_sampler,
            collate_fn=partial(
                prepare_train_input, 
                pad_idx=args.bos_idx),
                num_workers=0)

        data_loaders.append(data_loader)

    return data_loaders

def prepare_train_input(insts, pad_idx):
    # 通过paddlenlp.data.Pad来padding，用于对齐同一batch中样本的长度
    word_pad = Pad(pad_idx)
    src_word = word_pad([inst[0] for inst in insts])
    trg_word = word_pad(inst[1][:-1] for inst in insts)
    lbl_word = word_pad([inst[1][1:] for inst in insts])
    data_inputs = [src_word, trg_word, lbl_word]

    return data_inputs


In [7]:
# 为测试集创建dataloader（同训练集和验证集的dataloader）
def create_infer_loader(args, places=None):
    data_files = {'test': args.predict_file, }
    dataset = load_dataset(
        read, src_tgt_file=data_files['test'], only_src=True, lazy=False)

    src_vocab = Vocab.load_vocabulary(
        args.src_vocab_fpath,
        bos_token=args.special_token[0],
        eos_token=args.special_token[1],
        unk_token=args.special_token[2])

    trg_vocab = Vocab.load_vocabulary(
        args.trg_vocab_fpath,
        bos_token=args.special_token[0],
        eos_token=args.special_token[1],
        unk_token=args.special_token[2])

    args.src_vocab_size = len(src_vocab)
    args.trg_vocab_size = len(trg_vocab)

    def convert_samples(sample):
        source = sample['src'].split()
        source = src_vocab.to_indices(source) + [args.eos_idx]
        target = [args.bos_idx]
        return source, target
        
    dataset = dataset.map(convert_samples, lazy=False)

    # BatchSampler: https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/io/BatchSampler_cn.html
    batch_sampler = BatchSampler(dataset,batch_size=args.batch_size,drop_last=False)

    # DataLoader: https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/io/DataLoader_cn.html
    data_loader = DataLoader(
        dataset=dataset,
        places=places,
        batch_sampler=batch_sampler,
        collate_fn=partial(
            prepare_infer_input, 
            pad_idx=args.bos_idx),
            num_workers=0,
            return_list=True)

    return data_loader, trg_vocab.to_tokens

def prepare_infer_input(insts, pad_idx):
    """
    Put all padded data needed by beam search decoder into a list.
    """
    word_pad = Pad(pad_idx)
    src_word = word_pad(inst[0] for inst in insts)

    return [src_word, ]

## 3. 搭建模型
基于PaddlePaddle的相关API，包括：  
* [`paddle.nn.TransformerEncoderLayer`](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/TransformerEncoderLayer_cn.html#transformerencoderlayer)：Transformer编码器层
* [`paddle.nn.TransformerEncoder`](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/TransformerEncoder_cn.html#transformerencoder)：Transformer编码器
* [`paddle.nn.TransformerDecoderLayer`](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/TransformerDecoderLayer_cn.html#transformerdecoderlayer)：Transformer解码器层
* [`paddle.nn.TransformerDecoder`](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/TransformerDecoder_cn.html#transformerdecoder)：Transformer解码器


<center><img src="https://ai-studio-static-online.cdn.bcebos.com/31886c6aa0424fce92fb9a78248bd7d252f843bfa1f34e5f91bc763c62b46c29" width="700" height="350" ></center>
<br><center>图10：模型搭建 </center></br>

### Encoder层
和原生Transformer一致

### Decoder层
基于paddle.nn.TransformerDecoderLayer加入Wait-k策略

In [8]:
# 定义Decoder层，这里给出和nn.TransformerDecoderLayer不一致地方的注释
class DecoderLayer(nn.TransformerDecoderLayer):
    def __init__(self, *args, **kwargs):
        super(DecoderLayer, self).__init__(*args, **kwargs)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None, cache=None):
        residual = tgt
        # 计算LayerNorm
        if self.normalize_before:
            tgt = self.norm1(tgt)
        # 计算self attention
        if cache is None:
            tgt = self.self_attn(tgt, tgt, tgt, tgt_mask, None)
        else:
            tgt, incremental_cache = self.self_attn(tgt, tgt, tgt, tgt_mask,
                                                    cache[0])
                                                
        # 加上残差
        tgt = residual + self.dropout1(tgt)
        if not self.normalize_before:
            tgt = self.norm1(tgt)

        residual = tgt
        # 计算LayerNorm
        if self.normalize_before:
            tgt = self.norm2(tgt)

        # 下面是添加了waitk策略的部分
        # memory为encoder的output
        if len(memory) == 1:
            # 整句模型
            tgt = self.cross_attn(tgt, memory[0], memory[0], memory_mask, None)        
        else:
            # Wait-k策略
            cross_attn_outputs = []
            for i in range(tgt.shape[1]):
                # 取当前target第i位
                q = tgt[:, i:i + 1, :]
                if i >= len(memory):
                    e = memory[-1]
                else:
                    e = memory[i]
                # 计算cross attention
                cross_attn_outputs.append(
                    self.cross_attn(q, e, e, memory_mask[:, :, i:i + 1, :
                                                         e.shape[1]], None))
            # 将target每一位计算出的cross attention进行拼接
            tgt = paddle.concat(cross_attn_outputs, axis=1)

        # 加上残差
        tgt = residual + self.dropout2(tgt)
        if not self.normalize_before:
            tgt = self.norm2(tgt)

        residual = tgt
        # 计算LayerNorm
        if self.normalize_before:
            tgt = self.norm3(tgt)
        tgt = self.linear2(self.dropout(self.activation(self.linear1(tgt))))
        # 加上残差
        tgt = residual + self.dropout3(tgt)
        if not self.normalize_before:
            tgt = self.norm3(tgt)
        return tgt if cache is None else (tgt, (incremental_cache, ))

### 模型主结构
和原生Transformer基本一致，具体细节可参考可参考：[paddlenlp.transformers.TransformerModel](https://github.com/PaddlePaddle/PaddleNLP/blob/develop/paddlenlp/transformers/transformer/modeling.py)

SimultaneousTransformer：Encoder+Decoder(wait-k 策略)

### Transformer
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/4b3299f4552e4f82b93707d91e765fdbb617ab5fbb0c495aa769059c1b3c8ec2" width="1200" height="600" ></center>

### SimultaneousTransformer
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/7a46a5bced474d02b20c23685e4534c727462ea007f44e52922fa701da3a423a" width="1200" height="600" ></center>  

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/1628f330ab73442496c17c8a2ecd3276719e521776474b97ba20277c58bf04ea" width="1200" height="600" ></center>  

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/830c3250c6b546f6bce6f75148286c9f7bd7c17f6b8d40958c783bd2b0e47b81" width="1200" height="600" ></center>  


<center><img src="https://ai-studio-static-online.cdn.bcebos.com/8cf8837e63ee4cf593fb2327886574134f3ce9febb19446c8600a1608b36b6fd" width="1200" height="600" ></center>  

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/29729d66acab4e909fb107cb80be8ccd60a2511eeee64f75aae60937bf0ae4f8" width="1200" height="600" ></center>  

<br><center>图11：Example </center></br>


In [9]:

# 定义SimultaneousTransformer，这里给出和nn.TransformerDecoderLayer不一致地方的注释
class SimultaneousTransformer(nn.Layer):
    """
    model
    """

    def __init__(self,
                 src_vocab_size,
                 trg_vocab_size,
                 max_length,
                 n_layer,
                 n_head,
                 d_model,
                 d_inner_hid,
                 dropout,
                 weight_sharing,
                 bos_id=0,
                 eos_id=1,
                 waitk=-1):
        super(SimultaneousTransformer, self).__init__()
        self.trg_vocab_size = trg_vocab_size
        self.emb_dim = d_model
        self.bos_id = bos_id
        self.eos_id = eos_id
        self.dropout = dropout
        self.waitk = waitk
        self.n_layer = n_layer
        self.n_head = n_head
        self.d_model = d_model
        
        # 声明WordEmbedding
        self.src_word_embedding = WordEmbedding(
            vocab_size=src_vocab_size, emb_dim=d_model, bos_id=self.bos_id)

        # 声明PositionalEmbedding
        self.src_pos_embedding = PositionalEmbedding(
            emb_dim=d_model, max_length=max_length)
        
        # 判断target是否要和source共享WordEmbedding
        if weight_sharing:
            assert src_vocab_size == trg_vocab_size, (
                "Vocabularies in source and target should be same for weight sharing."
            )
            self.trg_word_embedding = self.src_word_embedding
            self.trg_pos_embedding = self.src_pos_embedding
        else:
            self.trg_word_embedding = WordEmbedding(
                vocab_size=trg_vocab_size, emb_dim=d_model, bos_id=self.bos_id)
            self.trg_pos_embedding = PositionalEmbedding(
                emb_dim=d_model, max_length=max_length)

        # 声明Encoder层
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_head,
            dim_feedforward=d_inner_hid,
            dropout=dropout,
            activation='relu',
            normalize_before=True,
            bias_attr=[False, True])
        encoder_norm = nn.LayerNorm(d_model)
        # 声明Encoder
        self.encoder = nn.TransformerEncoder(
            encoder_layer=encoder_layer, num_layers=n_layer, norm=encoder_norm)

        # 声明Decoder层
        decoder_layer = DecoderLayer(
            d_model=d_model,
            nhead=n_head,
            dim_feedforward=d_inner_hid,
            dropout=dropout,
            activation='relu',
            normalize_before=True,
            bias_attr=[False, False, True])
        decoder_norm = nn.LayerNorm(d_model)
        # 声明Decoder
        self.decoder = Decoder(
            decoder_layer=decoder_layer, num_layers=n_layer, norm=decoder_norm)

        if weight_sharing:
            self.linear = lambda x: paddle.matmul(
                x=x, y=self.trg_word_embedding.word_embedding.weight, transpose_y=True)
        else:
            self.linear = nn.Linear(
                in_features=d_model,
                out_features=trg_vocab_size,
                bias_attr=False)

    def forward(self, src_word, trg_word):
        src_max_len = paddle.shape(src_word)[-1]
        trg_max_len = paddle.shape(trg_word)[-1]
        base_attn_bias = paddle.cast(
            src_word == self.bos_id,
            dtype=paddle.get_default_dtype()).unsqueeze([1, 2]) * -1e9
        # 计算source端的attention mask
        src_slf_attn_bias = base_attn_bias
        src_slf_attn_bias.stop_gradient = True
        # 计算target端的attention mask
        trg_slf_attn_bias = paddle.tensor.triu(
            (paddle.ones(
                (trg_max_len, trg_max_len),
                dtype=paddle.get_default_dtype()) * -np.inf),
            1)
        trg_slf_attn_bias.stop_gradient = True
        # 计算encoder-decoder的attention mask
        trg_src_attn_bias = paddle.tile(base_attn_bias, [1, 1, trg_max_len, 1])
        src_pos = paddle.cast(
            src_word != self.bos_id, dtype="int64") * paddle.arange(
                start=0, end=src_max_len)
        trg_pos = paddle.cast(
            trg_word != self.bos_id, dtype="int64") * paddle.arange(
                start=0, end=trg_max_len)
        # 计算source的word embedding
        src_emb = self.src_word_embedding(src_word)
        # 计算source的position embedding
        src_pos_emb = self.src_pos_embedding(src_pos)
        # 得到最终Embedding：word embedding + position embedding
        src_emb = src_emb + src_pos_emb
        enc_input = F.dropout(
            src_emb, p=self.dropout,
            training=self.training) if self.dropout else src_emb
        with paddle.static.amp.fp16_guard():
            # 下面是添加了waitk策略的部分
            if self.waitk >= src_max_len or self.waitk == -1:
                # 整句模型，和API一致
                enc_outputs = [
                    self.encoder(
                        enc_input, src_mask=src_slf_attn_bias)
                ]
            else:
                # Wait-k策略
                enc_outputs = []
                for i in range(self.waitk, src_max_len + 1):
                    # 分别将子句送入encoder
                    enc_output = self.encoder(
                        enc_input[:, :i, :],
                        src_mask=src_slf_attn_bias[:, :, :, :i])
                    enc_outputs.append(enc_output)
            # 计算target的word embedding
            trg_emb = self.trg_word_embedding(trg_word)
            # 计算target的position embedding
            trg_pos_emb = self.trg_pos_embedding(trg_pos)
            # 得到最终Embedding：word embedding + position embedding
            trg_emb = trg_emb + trg_pos_emb
            dec_input = F.dropout(
                trg_emb, p=self.dropout,
                training=self.training) if self.dropout else trg_emb
            # 送入Decoder，拿到输出
            dec_output = self.decoder(
                dec_input,
                enc_outputs,
                tgt_mask=trg_slf_attn_bias,
                memory_mask=trg_src_attn_bias)
            # 经过全连接层拿到最终输出
            predict = self.linear(dec_output)

        return predict

    def greedy_search(self, src_word, max_len=256, waitk=-1):
        src_max_len = paddle.shape(src_word)[-1]
        base_attn_bias = paddle.cast(
            src_word == self.bos_id,
            dtype=paddle.get_default_dtype()).unsqueeze([1, 2]) * -1e9
        # 计算source端的attention mask
        src_slf_attn_bias = base_attn_bias
        src_slf_attn_bias.stop_gradient = True
        # 计算target端的attention mask
        trg_src_attn_bias = paddle.tile(base_attn_bias, [1, 1, 1, 1])
        src_pos = paddle.cast(
            src_word != self.bos_id, dtype="int64") * paddle.arange(
                start=0, end=src_max_len)
        # 计算source的word embedding
        src_emb = self.src_word_embedding(src_word)
        # 计算source的position embedding
        src_pos_emb = self.src_pos_embedding(src_pos)
        # 得到最终Embedding：word embedding + position embedding
        src_emb = src_emb + src_pos_emb
        enc_input = F.dropout(
            src_emb, p=self.dropout,
            training=self.training) if self.dropout else src_emb
        # 下面是添加了waitk策略的部分
        if waitk < 0 or waitk > src_max_len:
            # 整句模型
            enc_outputs = [self.encoder(enc_input, src_mask=src_slf_attn_bias)]
        else:
            # waitk策略
            enc_outputs = []
            # 依次将source子句送入Encoder，拿到输出
            for i in range(waitk, src_max_len + 1):
                enc_output = self.encoder(
                    enc_input[:, :i, :],
                    src_mask=src_slf_attn_bias[:, :, :, :i])
                enc_outputs.append(enc_output)

        batch_size = enc_outputs[-1].shape[0]
        max_len = (
            enc_outputs[-1].shape[1] + 20) if max_len is None else max_len
        end_token_tensor = paddle.full(
            shape=[batch_size, 1], fill_value=self.eos_id, dtype="int64")

        predict_ids = []
        # 初始化概率
        log_probs = paddle.full(
            shape=[batch_size, 1], fill_value=0, dtype="float32")
        # 初始化trg_word
        trg_word = paddle.full(
            shape=[batch_size, 1], fill_value=self.bos_id, dtype="int64")

        # 初始化caches：包括StaticCache和IncrementalCache
        caches = self.decoder.gen_cache(enc_outputs[-1], do_zip=False)

        for i in range(max_len):
            trg_pos = paddle.full(
                shape=trg_word.shape, fill_value=i, dtype="int64")
            # 计算target的word embedding
            trg_emb = self.trg_word_embedding(trg_word)
            # 计算target的position embedding
            trg_pos_emb = self.trg_pos_embedding(trg_pos)
            # 得到最终Embedding：word embedding + position embedding
            trg_emb = trg_emb + trg_pos_emb
            dec_input = F.dropout(
                trg_emb, p=self.dropout,
                training=self.training) if self.dropout else trg_emb

            if waitk < 0 or i >= len(enc_outputs):
                # 整句模型
                _e = enc_outputs[-1]
                dec_output, caches = self.decoder(
                    dec_input, [_e], None,
                    trg_src_attn_bias[:, :, :, :_e.shape[1]], caches)
            else:
                _e = enc_outputs[i]
                # 每次将子句的encoder输出送进去解码
                dec_output, caches = self.decoder(
                    dec_input, [_e], None,
                    trg_src_attn_bias[:, :, :, :_e.shape[1]], caches)

            dec_output = paddle.reshape(
                dec_output, shape=[-1, dec_output.shape[-1]])

            logits = self.linear(dec_output)
            # 拿到当前这一步的概率，并加到上一步的概率上
            step_log_probs = paddle.log(F.softmax(logits, axis=-1))
            log_probs = paddle.add(x=step_log_probs, y=log_probs)
            scores = log_probs
            # 获取得分最高的概率和下标
            topk_scores, topk_indices = paddle.topk(x=scores, k=1)
            
            # 判断是否遇到结束符
            finished = paddle.equal(topk_indices, end_token_tensor)
            # 更新trg_word
            trg_word = topk_indices
            # 更新概率log_probs
            log_probs = topk_scores
            
            # 将结果保留到predict_ids
            predict_ids.append(topk_indices)

            if paddle.all(finished).numpy():
                break
        
        # 将predict_ids里面的结果堆叠成Tensor
        predict_ids = paddle.stack(predict_ids, axis=0)
        finished_seq = paddle.transpose(predict_ids, [1, 2, 0])
        finished_scores = topk_scores

        return finished_seq, finished_scores

## 4.训练模型
运行`do_train`函数，
在`do_train`函数中，配置优化器、损失函数，以及评价指标（Perplexity，即困惑度，常用来衡量语言模型优劣，也可用于机器翻译、文本生成等任务）。
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/46df720d17b64f93a64ff52fa943219dea54c53c1d7f47c8a79f482c39fa7201" width="600" height="300" ></center>
<br><center>图12：训练模型 </center></br>


In [10]:
# 读入参数
yaml_file = 'transformer.yaml'
with open(yaml_file, 'rt') as f:
    args = AttrDict(yaml.safe_load(f))
    pprint(args)

{'batch_size': 10,
 'beam_size': 1,
 'beta1': 0.9,
 'beta2': 0.997,
 'bos_idx': 0,
 'd_inner_hid': 2048,
 'd_model': 512,
 'device': 'gpu',
 'dropout': 0.1,
 'eos_idx': 1,
 'epoch': 1,
 'eps': '1e-9',
 'init_from_params': 'trained_models/nist_zhen_full_w5/',
 'label_smooth_eps': 0.1,
 'learning_rate': 2.0,
 'max_length': 256,
 'max_out_len': 256,
 'n_best': 1,
 'n_head': 8,
 'n_layer': 6,
 'output_file': 'train_dev_test/predict.txt',
 'predict_file': 'train_dev_test/test_08.zh.bpe',
 'print_step': 10,
 'random_seed': 42,
 'save_model': 'trained_models',
 'save_step': 20,
 'shuffle': True,
 'shuffle_batch': True,
 'special_token': ['<s>', '<e>', '<unk>'],
 'src_vocab_fpath': 'train_dev_test/nist.20k.zh.vocab',
 'src_vocab_size': 10000,
 'training_file': 'train_dev_test/demo.train.zhen.bpe',
 'trg_vocab_fpath': 'train_dev_test/nist.10k.en.vocab',
 'trg_vocab_size': 10000,
 'unk_idx': 2,
 'use_amp': False,
 'validation_file': 'train_dev_test/demo.dev.zhen.bpe',
 'waitk': 5,
 'warmup_steps

In [11]:
def do_train(args):
    # 设置在GPU/CPU/XPU上运行
    paddle.set_device(args.device)

    # 设置随机种子
    random_seed = eval(str(args.random_seed))
    if random_seed is not None:
        paddle.seed(random_seed)

    # 获取Dataloader
    (train_loader), (eval_loader) = create_data_loader(
        args, places=paddle.get_device())

    # 声明模型
    transformer = SimultaneousTransformer(
        args.src_vocab_size, args.trg_vocab_size, args.max_length + 1,
        args.n_layer, args.n_head, args.d_model, args.d_inner_hid, args.dropout,
        args.weight_sharing, args.bos_idx, args.eos_idx, args.waitk)
    

    print('waitk=', args.waitk)

    # 定义Loss
    criterion = CrossEntropyCriterion(args.label_smooth_eps, args.bos_idx)

    # 定义学习率的衰减策略
    scheduler = paddle.optimizer.lr.NoamDecay(args.d_model, args.warmup_steps,
                                              args.learning_rate)
    # 定义优化器
    optimizer = paddle.optimizer.Adam(
        learning_rate=scheduler,
        beta1=args.beta1,
        beta2=args.beta2,
        epsilon=float(args.eps),
        parameters=transformer.parameters())

    step_idx = 0

    # 按epoch迭代训练
    for pass_id in range(args.epoch):
        batch_id = 0
        for input_data in train_loader:
            # 从训练集Dataloader按batch取数据
            (src_word, trg_word, lbl_word) = input_data
      
            # 获得模型输出的logits 
            logits = transformer(src_word=src_word, trg_word=trg_word)

            # 计算loss
            sum_cost, avg_cost, token_num = criterion(logits, lbl_word)

            # 计算梯度
            avg_cost.backward() 
            # 更新参数
            optimizer.step() 
            # 梯度清零
            optimizer.clear_grad() 

            if (step_idx + 1) % args.print_step == 0 or step_idx == 0:
                total_avg_cost = avg_cost.numpy()
                # 打印log
                logger.info(
                    "step_idx: %d, epoch: %d, batch: %d, avg loss: %f, "
                    " ppl: %f " %
                    (step_idx, pass_id, batch_id, total_avg_cost,
                        np.exp([min(total_avg_cost, 100)])))

            if (step_idx + 1) % args.save_step == 0:
                # 验证
                transformer.eval()
                total_sum_cost = 0
                total_token_num = 0
                with paddle.no_grad():
                    for input_data in eval_loader:
                        # 从验证集Dataloader按batch取数据
                        (src_word, trg_word, lbl_word) = input_data
                        # 获得模型输出的logits 
                        logits = transformer(
                            src_word=src_word, trg_word=trg_word)
                        # 计算loss
                        sum_cost, avg_cost, token_num = criterion(logits,
                                                                  lbl_word)
                        total_sum_cost += sum_cost.numpy()
                        total_token_num += token_num.numpy()
                        total_avg_cost = total_sum_cost / total_token_num
                    # 打印log
                    logger.info("validation, step_idx: %d, avg loss: %f, "
                                " ppl: %f" %
                                (step_idx, total_avg_cost,
                                 np.exp([min(total_avg_cost, 100)])))
                transformer.train()

                if args.save_model:
                    # 保存中间模型
                    model_dir = os.path.join(args.save_model,
                                             "step_" + str(step_idx))
                    if not os.path.exists(model_dir):
                        os.makedirs(model_dir)
                    # 保存模型参数
                    paddle.save(transformer.state_dict(),
                                os.path.join(model_dir, "transformer.pdparams"))
                    # 保存优化器参数
                    paddle.save(optimizer.state_dict(),
                                os.path.join(model_dir, "transformer.pdopt"))

            batch_id += 1
            step_idx += 1
            scheduler.step()

    if args.save_model:
        # 保存最终模型
        model_dir = os.path.join(args.save_model, "step_final")
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        # 保存模型参数
        paddle.save(transformer.state_dict(),
                    os.path.join(model_dir, "transformer.pdparams"))
        # 保存优化器参数
        paddle.save(optimizer.state_dict(),
                    os.path.join(model_dir, "transformer.pdopt"))

In [12]:
do_train(args)

waitk= 5


[2021-06-21 22:24:29,342] [    INFO] - step_idx: 0, epoch: 0, batch: 0, avg loss: 9.265658,  ppl: 10568.766602 
[2021-06-21 22:24:59,984] [    INFO] - step_idx: 9, epoch: 0, batch: 9, avg loss: 9.233420,  ppl: 10233.484375 
[2021-06-21 22:25:27,962] [    INFO] - step_idx: 19, epoch: 0, batch: 19, avg loss: 9.175573,  ppl: 9658.304688 
[2021-06-21 22:25:43,529] [    INFO] - validation, step_idx: 19, avg loss: 9.170717,  ppl: 9611.516602


## 5. 预测和评估
模型最终训练的效果一般可通过测试集来进行测试，同传类似机器翻译场景，一般计算BLEU值。
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/e0ab4ed332f54e62b5a203e20e599b450e2b97b23e674534b582d65b0e5ac6b4" width="600" height="300" ></center>
<br><center>图13： 预测和评估 </center></br>

In [13]:

def do_predict(args):
    
    paddle.set_device(args.device)

    # 获取Dataloader
    test_loader, to_tokens = create_infer_loader(args)

    # 声明模型
    transformer = SimultaneousTransformer(
        args.src_vocab_size, args.trg_vocab_size, args.max_length + 1,
        args.n_layer, args.n_head, args.d_model, args.d_inner_hid, args.dropout,
        args.weight_sharing, args.bos_idx, args.eos_idx, args.waitk)

    # 加载预训练模型
    assert args.init_from_params, (
        "Please set init_from_params to load the infer model.")

    model_dict = paddle.load(
        os.path.join(args.init_from_params, "transformer.pdparams"))

    # 避免长度大于训练时候设定的长度，这里重新设定 
    model_dict["src_pos_embedding.pos_encoder.weight"] = position_encoding_init(
        args.max_length + 1, args.d_model)
    model_dict["trg_pos_embedding.pos_encoder.weight"] = position_encoding_init(
        args.max_length + 1, args.d_model)

    transformer.load_dict(model_dict)

    # 设置评估模式
    transformer.eval()

    f = open(args.output_file, "w", encoding='utf8')

    with paddle.no_grad():
        for input_data in test_loader:
            
            (src_word, ) = input_data

            finished_seq, finished_scores = transformer.greedy_search(
                src_word, max_len=args.max_out_len, waitk=args.waitk)
            finished_seq = finished_seq.numpy()
            finished_scores = finished_scores.numpy()
            for idx, ins in enumerate(finished_seq):
                for beam_idx, beam in enumerate(ins):
                    if beam_idx >= args.n_best:
                        break
                    id_list = post_process_seq(beam, args.bos_idx, args.eos_idx)
                    word_list = to_tokens(id_list)
                    sequence = ' '.join(word_list) + "\n"
                    f.write(sequence)
    f.close()

In [14]:
do_predict(args)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlenlp/data/vocab.py:217: UserWarning: The type of `to_tokens()`'s input `indices` is not `int` which will be forcibly transfered to `int`. 
  "The type of `to_tokens()`'s input `indices` is not `int` which will be forcibly transfered to `int`. "


### 模型评估
预测结果中每行输出是对应行输入的得分最高的翻译，对于使用 BPE 的数据，预测出的翻译结果也将是 BPE 表示的数据，要还原成原始的数据（这里指 tokenize 后的数据）才能进行正确的评估

In [15]:
# 还原 predict.txt 中的预测结果为 tokenize 后的数据
! sed -r 's/(@@ )|(@@ ?$)//g' train_dev_test/predict.txt > train_dev_test/predict.tok.txt
# BLEU评估工具来源于 https://github.com/moses-smt/mosesdecoder.git
! tar -zxf mosesdecoder.tar.gz
#计算multi-bleu
! perl mosesdecoder/scripts/generic/multi-bleu.perl train_dev_test/test_08.en.* < train_dev_test/predict.tok.txt

BLEU = 36.29, 72.0/45.2/28.8/18.5 (BP=1.000, ratio=1.025, hyp_len=23631, ref_len=23049)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
